# OpenAPI仕様を使った関数呼び出し

インターネットの多くはRESTful APIによって動作しています。GPTにAPIを呼び出す能力を与えることで、可能性の世界が広がります。このノートブックでは、GPTがAPIを知的に呼び出すために使用できる方法を実演します。OpenAPI仕様と連鎖関数呼び出しを活用しています。

[OpenAPI仕様（OAS）](https://swagger.io/specification/)は、RESTful APIの詳細を機械が読み取り解釈できる形式で記述するための、普遍的に受け入れられた標準です。これにより、人間とコンピュータの両方がサービスの機能を理解できるようになり、GPTにAPIの呼び出し方法を示すために活用することができます。

このノートブックは2つの主要なセクションに分かれています：

1. サンプルのOpenAPI仕様をチャット補完APIの関数定義リストに変換する方法
2. チャット補完APIを使用して、ユーザーの指示に基づいてこれらの関数を知的に呼び出す方法

続行する前に、[関数呼び出し](./How_to_call_functions_with_chat_models.ipynb)について理解を深めることをお勧めします。

In [1]:
!pip install -q jsonref # for resolving $ref's in the OpenAPI spec
!pip install -q openai

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [7]:
import os
import json
import jsonref
from openai import OpenAI
import requests
from pprint import pp

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

## OpenAPI仕様を関数定義に変換する方法

ここで使用するOpenAPIスペックの例は`gpt-4`を使用して作成されました。このサンプルスペックを、チャット補完APIに提供できる関数定義のセットに変換します。モデルは、提供されたユーザー指示に基づいて、これらの関数を呼び出すために必要な引数を含むJSONオブジェクトを生成します。

進める前に、この生成されたスペックを確認してみましょう。OpenAPIスペックには、APIのエンドポイント、それらがサポートする操作、受け入れるパラメータ、処理できるリクエスト、返すレスポンスの詳細が含まれています。スペックはJSON形式で定義されています。

スペック内のエンドポイントには以下の操作が含まれています：

- すべてのイベントの一覧表示
- 新しいイベントの作成
- IDによるイベントの取得
- IDによるイベントの削除
- IDによるイベント名の更新

スペック内の各操作には`operationId`があり、スペックを関数仕様に解析する際に関数名として使用します。スペックには、各操作のパラメータのデータ型と構造を定義するスキーマも含まれています。

スキーマはこちらで確認できます：

In [3]:
with open('./data/example_events_openapi.json', 'r') as f:
    openapi_spec = jsonref.loads(f.read()) # it's important to load with jsonref, as explained below

display(openapi_spec)

{'openapi': '3.0.0',
 'info': {'version': '1.0.0',
  'title': 'Event Management API',
  'description': 'An API for managing event data'},
 'paths': {'/events': {'get': {'summary': 'List all events',
    'operationId': 'listEvents',
    'responses': {'200': {'description': 'A list of events',
      'content': {'application/json': {'schema': {'type': 'array',
         'items': {'type': 'object',
          'properties': {'id': {'type': 'string'},
           'name': {'type': 'string'},
           'date': {'type': 'string', 'format': 'date-time'},
           'location': {'type': 'string'}},
          'required': ['name', 'date', 'location']}}}}}}},
   'post': {'summary': 'Create a new event',
    'operationId': 'createEvent',
    'requestBody': {'required': True,
     'content': {'application/json': {'schema': {'type': 'object',
        'properties': {'id': {'type': 'string'},
         'name': {'type': 'string'},
         'date': {'type': 'string', 'format': 'date-time'},
         'location

OpenAPI仕様について十分に理解できたので、これを関数仕様に解析することができます。

シンプルな`openapi_to_functions`関数を書いて、定義のリストを生成できます。各関数は以下のキーを含む辞書として表現されます：

- `name`: これはOpenAPI仕様で定義されたAPIエンドポイントのオペレーション識別子に対応します。
- `description`: これは関数の簡潔な説明または要約で、関数が何をするかの概要を提供します。
- `parameters`: これは関数の期待される入力パラメータを定義するスキーマです。各パラメータの型、必須かオプションか、その他の関連する詳細についての情報を提供します。

スキーマで定義された各エンドポイントについて、以下を行う必要があります：

1. **JSON参照の解決**: OpenAPI仕様では、重複を避けるためにJSON参照（$refとも呼ばれる）を使用することが一般的です。これらの参照は、複数の場所で使用される定義を指します。例えば、複数のAPIエンドポイントが同じオブジェクト構造を返す場合、その構造を一度定義してから、必要な場所で参照することができます。これらの参照を解決し、それらが指すコンテンツに置き換える必要があります。

2. **関数の名前を抽出**: 単純にoperationIdを関数名として使用します。代替として、エンドポイントのパスとオペレーションを関数名として使用することもできます。

3. **説明とパラメータを抽出**: `description`、`summary`、`requestBody`、`parameters`フィールドを反復処理して、関数の説明とパラメータを設定します。

実装は以下の通りです：

In [9]:
def openapi_to_functions(openapi_spec):
    functions = []

    for path, methods in openapi_spec["paths"].items():
        for method, spec_with_ref in methods.items():
            # 1. Resolve JSON references.
            spec = jsonref.replace_refs(spec_with_ref)

            # 2. Extract a name for the functions.
            function_name = spec.get("operationId")

            # 3. Extract a description and parameters.
            desc = spec.get("description") or spec.get("summary", "")

            schema = {"type": "object", "properties": {}}

            req_body = (
                spec.get("requestBody", {})
                .get("content", {})
                .get("application/json", {})
                .get("schema")
            )
            if req_body:
                schema["properties"]["requestBody"] = req_body

            params = spec.get("parameters", [])
            if params:
                param_properties = {
                    param["name"]: param["schema"]
                    for param in params
                    if "schema" in param
                }
                schema["properties"]["parameters"] = {
                    "type": "object",
                    "properties": param_properties,
                }

            functions.append(
                {"type": "function", "function": {"name": function_name, "description": desc, "parameters": schema}}
            )

    return functions


functions = openapi_to_functions(openapi_spec)

for function in functions:
    pp(function)
    print()


{'type': 'function',
 'function': {'name': 'listEvents',
              'description': 'List all events',
              'parameters': {'type': 'object', 'properties': {}}}}

{'type': 'function',
 'function': {'name': 'createEvent',
              'description': 'Create a new event',
              'parameters': {'type': 'object',
                             'properties': {'requestBody': {'type': 'object',
                                                            'properties': {'id': {'type': 'string'},
                                                                           'name': {'type': 'string'},
                                                                           'date': {'type': 'string',
                                                                                    'format': 'date-time'},
                                                                           'location': {'type': 'string'}},
                                                            'required':

## GPTでこれらの関数を呼び出す方法

これらの関数定義を用意したので、ユーザーの入力に基づいてGPTがそれらを適切に呼び出せるように活用できます。

重要な点として、chat completions APIは関数を実行するのではなく、あなた自身のコードで関数を呼び出すために使用できるJSONを生成することに注意してください。

関数呼び出しの詳細については、専用の[関数呼び出しガイド](./How_to_call_functions_with_chat_models.ipynb)を参照してください。

In [23]:
SYSTEM_MESSAGE = """
You are a helpful assistant.
Respond to the following prompt by using function_call and then summarize actions.
Ask for clarification if a user request is ambiguous.
"""

# Maximum number of function calls allowed to prevent infinite or lengthy loops
MAX_CALLS = 5


def get_openai_response(functions, messages):
    return client.chat.completions.create(
        model="gpt-3.5-turbo-16k",
        tools=functions,
        tool_choice="auto",  # "auto" means the model can pick between generating a message or calling a function.
        temperature=0,
        messages=messages,
    )


def process_user_instruction(functions, instruction):
    num_calls = 0
    messages = [
        {"content": SYSTEM_MESSAGE, "role": "system"},
        {"content": instruction, "role": "user"},
    ]

    while num_calls < MAX_CALLS:
        response = get_openai_response(functions, messages)
        message = response.choices[0].message
        print(message)
        try:
            print(f"\n>> Function call #: {num_calls + 1}\n")
            pp(message.tool_calls)
            messages.append(message)

            # For the sake of this example, we'll simply add a message to simulate success.
            # Normally, you'd want to call the function here, and append the results to messages.
            messages.append(
                {
                    "role": "tool",
                    "content": "success",
                    "tool_call_id": message.tool_calls[0].id,
                }
            )

            num_calls += 1
        except:
            print("\n>> Message:\n")
            print(message.content)
            break

    if num_calls >= MAX_CALLS:
        print(f"Reached max chained function calls: {MAX_CALLS}")


USER_INSTRUCTION = """
Instruction: Get all the events.
Then create a new event named AGI Party.
Then delete event with id 2456.
"""

process_user_instruction(functions, USER_INSTRUCTION)


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_jmlvEyMRMvOtB80adX9RbqIV', function=Function(arguments='{}', name='listEvents'), type='function')])

>> Function call #: 1

[ChatCompletionMessageToolCall(id='call_jmlvEyMRMvOtB80adX9RbqIV', function=Function(arguments='{}', name='listEvents'), type='function')]
ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_OOPOY7IHMq3T7Ib71JozlUQJ', function=Function(arguments='{\n  "requestBody": {\n    "id": "1234",\n    "name": "AGI Party",\n    "date": "2022-12-31",\n    "location": "New York"\n  }\n}', name='createEvent'), type='function')])

>> Function call #: 2

[ChatCompletionMessageToolCall(id='call_OOPOY7IHMq3T7Ib71JozlUQJ', function=Function(arguments='{\n  "requestBody": {\n    "id": "1234",\n    "name": "AGI Party",\n    "date": "2022-12-31",\n    "location": "New York"\n  }\n}', name='c

### 結論

OpenAPI仕様をGPTが知的に呼び出すことができる関数仕様に変換する方法を実演し、これらを連鎖させて複雑な操作を実行する方法を示しました。

このシステムの可能な拡張には、条件分岐やループを必要とするより複雑なユーザー指示の処理、実際の操作を実行するための実際のAPIとの統合、指示が実行可能で関数呼び出しが成功することを保証するためのエラーハンドリングと検証の改善などが含まれます。